In [ ]:
import nltk
import numpy as np
from gensim.models import Word2Vec, KeyedVectors
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min


In [ ]:
content = '''Chính phủ đề xuất tiếp tục thực hiện chính sách giảm thuế VAT từ 10% còn 8%, trừ một số nhóm hàng hóa, dịch vụ như ngân hàng, bảo hiểm, kinh doanh bất động sản…
Bộ trưởng Bộ Tài chính Hồ Đức Phớc chiều 24/5 trình bày Tờ trình về tiếp tục thực hiện chính sách giảm thuế giá trị gia tăng (VAT) 2% theo Nghị quyết số 43/2022/QH15 ngày 11/1/2022. Thời gian đề xuất áp dụng, theo tờ trình, là từ ngày 1/7 đến hết 31/12 năm nay.

Theo Bộ trưởng Phớc, thực tế cho thấy, các giải pháp hỗ trợ doanh nghiệp, người dân về thuế, phí và lệ phí nói chung và giảm thuế VAT nói riêng có tác động tích cực và được cộng đồng doanh nghiệp, người dân hưởng ứng, đánh giá cao, qua đó góp phần vào những kết quả tích cực trong phục hồi và phát triển của doanh nghiệp, người dân và nền kinh tế trong thời gian qua.

Do vậy, năm 2023 Chính phủ đề xuất tiếp tục thực hiện chính sách giảm thuế VAT 2% quy định tại điểm a khoản 1.1 Điều 3 Nghị quyết số 43/2022/QH15 về chính sách tài khóa, tiền tệ hỗ trợ Chương trình phục hồi và phát triển kinh tế - xã hội.

Cụ thể là giảm 2% thuế suất thuế giá trị gia tăng, áp dụng đối với các nhóm hàng hóa, dịch vụ đang áp dụng mức thuế suất 10% (còn 8%), trừ một số nhóm hàng hóa, dịch vụ sau: viễn thông, công nghệ thông tin, hoạt động tài chính, ngân hàng, chứng khoán, bảo hiểm, kinh doanh bất động sản, kim loại, sản phẩm từ kim loại đúc sẵn, sản phẩm khai khoáng (không kể khai thác than), than cốc, dầu mỏ tinh chế, sản phẩm hóa chất, sản phẩm hàng hóa và dịch vụ chịu thuế tiêu thụ đặc biệt.

Bộ trưởng Hồ Đức Phớc nhấn mạnh, thực hiện theo phương án này nhằm đảm bảo đúng mục tiêu kích cầu tiêu dùng, phù hợp với bối cảnh kinh tế hiện nay, qua đó thúc đẩy hoạt động sản xuất kinh doanh sớm phục hồi và phát triển để đóng góp trở lại cho ngân sách Nhà nước cũng như nền kinh tế.

Với giải pháp giảm thuế VAT cho các đối tượng như đã được thực hiện trong năm 2022 theo Nghị quyết số 43/2022/QH15, nếu áp dụng trong 6 tháng cuối năm nay thì dự kiến số giảm thu ngân sách Nhà nước tương đương khoảng 24.000 tỷ đồng (đối với thu ngân sách Nhà nước năm 2023 thì dự kiến giảm 20.000 tỷ đồng do số thu thuế VAT phải nộp của tháng 12/2023 nộp trong tháng 1/2024).

Bộ trưởng Hồ Đức Phớc khẳng định, người dân là đối tượng sẽ được hưởng lợi trực tiếp của chính sách này, việc giảm thuế VAT đối với hàng hóa, dịch vụ chịu thuế suất thuế VAT 10% sẽ góp phần giảm giá bán, từ đó góp phần giảm trực tiếp chi phí của người dân trong việc tiêu dùng hàng hóa, dịch vụ phục vụ đời sống nhân dân.

Đối với các doanh nghiệp sản xuất, kinh doanh hàng hóa, cung ứng dịch vụ chịu thuế VAT thuế suất 10% sẽ được hưởng lợi khi chính sách được ban hành. Việc giảm thuế VAT sẽ góp phần làm giảm chi phí sản xuất, hạ giá thành sản phẩm, từ đó giúp doanh nghiệp tăng khả năng phục hồi và mở rộng sản xuất kinh doanh.'''

In [133]:
contents_parsed = content.lower() #Biến đổi hết thành chữ thường
contents_parsed = contents_parsed.replace('\n', '. ') #Đổi các ký tự xuống dòng thành chấm câu
contents_parsed = contents_parsed.strip() #Loại bỏ đi các khoảng trắng thừa

In [134]:
!pip install nltk


In [135]:


nltk.download('punkt')
nltk.download('punkt_tab')  # bản mới của NLTK yêu cầu thêm gói này


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
sentences = nltk.sent_tokenize(contents_parsed)

In [138]:
w2v = KeyedVectors.load_word2vec_format("/content/drive/MyDrive/data/vi.vec")

In [139]:
!pip install pyvi

In [140]:
vocab = w2v.index_to_key #Danh sách các từ trong từ điển

from pyvi import ViTokenizer

X = []
for sentence in sentences:
    sentence_tokenized = ViTokenizer.tokenize(sentence)
    words = sentence_tokenized.split(" ")
    sentence_vec = np.zeros((100))
    for word in words:
        if word in vocab:
            sentence_vec += w2v[word]
    X.append(sentence_vec)


In [ ]:

n_clusters = 5
kmeans = KMeans(n_clusters=n_clusters)
kmeans = kmeans.fit(X)

In [ ]:
avg = []
for j in range(n_clusters):
    idx = np.where(kmeans.labels_ == j)[0]
    avg.append(np.mean(idx))
closest, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_, X)
ordering = sorted(range(n_clusters), key=lambda k: avg[k])
summary = ' '.join([sentences[closest[idx]] for idx in ordering])

In [143]:
summary

'. đối với các doanh nghiệp sản xuất, kinh doanh hàng hóa, cung ứng dịch vụ chịu thuế vat thuế suất 10% sẽ được hưởng lợi khi chính sách được ban hành. cụ thể là giảm 2% thuế suất thuế giá trị gia tăng, áp dụng đối với các nhóm hàng hóa, dịch vụ đang áp dụng mức thuế suất 10% (còn 8%), trừ một số nhóm hàng hóa, dịch vụ sau: viễn thông, công nghệ thông tin, hoạt động tài chính, ngân hàng, chứng khoán, bảo hiểm, kinh doanh bất động sản, kim loại, sản phẩm từ kim loại đúc sẵn, sản phẩm khai khoáng (không kể khai thác than), than cốc, dầu mỏ tinh chế, sản phẩm hóa chất, sản phẩm hàng hóa và dịch vụ chịu thuế tiêu thụ đặc biệt. do vậy, năm 2023 chính phủ đề xuất tiếp tục thực hiện chính sách giảm thuế vat 2% quy định tại điểm a khoản 1.1 điều 3 nghị quyết số 43/2022/qh15 về chính sách tài khóa, tiền tệ hỗ trợ chương trình phục hồi và phát triển kinh tế - xã hội.. . bộ trưởng hồ đức phớc khẳng định, người dân là đối tượng sẽ được hưởng lợi trực tiếp của chính sách này, việc giảm thuế vat đối